In [1]:
import random, re, csv
import numpy as np
from sklearn import metrics,svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from simdata import *
#from interdata_mysql_simple import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# for kmeans
from scipy import stats

In [2]:
#==============================================
# make mixdata from csv files
#==============================================
mixdata_sim_filedir='../resources/gamedata/mixdata_sim.csv'
interdata_sim_filedir='../resources/gamedata/interdata_sim.csv'
biodata_sim_filedir='../resources/gamedata/biodata_sim.csv'

# mixdata: [[interdata,biodata,prelabel],...,[]]
mixdata=np.asarray(get_mldata(mixdata_sim_filedir),dtype=float)

#mixdata=np.insert(raw_mixdata,[0],mixdata_indx,axis=1)

# interdata_features:4+1 biodata_feature:10+1
interdata=np.asarray(get_mldata(interdata_sim_filedir),dtype=float)
biodata=np.asarray(get_mldata(biodata_sim_filedir),dtype=float)



#==============================================
#prepare features/labels for k-means
#==============================================
mixdata_feature=mixdata[:,0:mixdata[0].shape[0]-1]
mixdata_feature_nd=(mixdata_feature - mixdata_feature.mean(axis=0)) / mixdata_feature.std(axis=0)
mixdata_prelabel=mixdata[:,mixdata[0].shape[0]-1]
#sample_indx=np.arange(raw_mixdata.shape[0]).reshape(raw_mixdata.shape[0],1)
#indx_mixdata_nd=np.insert(mixdata_feature_nd[0:2],[0],sample_indx[0:2],axis=1)





#================================
# Cluster from kmeans
#================================
#make train/test lists for building cluster
clu_train_feature,\
clu_test_feature,\
clu_train_label,\
clu_test_label = train_test_split(mixdata_feature_nd[:,0:4],mixdata_prelabel, test_size=.2)

# build cluster from k-means
cluster_km = KMeans(n_clusters=2, random_state=0).fit(clu_train_feature)

#================================
# Evaluation
#================================
#相似度检验
#kresult=cluster_km.predict(clu_train_feature)
#sprint("similarity test: ",metrics.adjusted_rand_score(kresult,clu_train_label))





#================================
# Classifier from SVM
#================================
#make train/test lists for building classifier
cluster_result=cluster_km.predict(mixdata_feature_nd[:,0:4])

clf_train_feature,\
clf_test_feature,\
clf_train_label,\
clf_test_label = train_test_split(mixdata_feature_nd[:,5:],cluster_result, test_size=.2)

# build classifier
classifier_svm=svm.SVC(C=1).fit(clf_train_feature,clf_train_label)

labels_pre_svm=classifier_svm.predict(clf_test_feature)


#============================
# Evaluation
#============================
# accuracy
acc_svm=accuracy_score(clf_test_label,labels_pre_svm)
print("acc_svm:",acc_svm)

acc_svm: 0.724137931034
